In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='DV'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [2]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")

In [3]:
from_tb = "OM_Delivery"

In [4]:
table_name = "sync_dms_dv"
table_temp = "sync_dms_dv_temp"

In [7]:
sql = \
f"""
DECLARE @from DATE = '{start_date}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,BatNbr,OrderNbr,Sequence) as pk,
BranchID,
BatNbr,
OrderNbr,
Sequence,
SlsperID,
Status,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime
from {from_tb}
where Crtd_Datetime >= @from
"""

In [8]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(crtd_datetime)
CLUSTER BY pk,branchid,batnbr,ordernbr
"""
execute_bq_query(sql)

1it [00:26, 26.44s/it]


In [9]:
bq_values_insert(df, f"{table_name}", 2)

1it [00:09,  9.78s/it]


In [ ]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
BatNbr,
OrderNbr,
Sequence,
SlsperID,
Status,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime
from {from_tb}
where LUpd_DateTime >= @from
and Crtd_DateTime < @from
"""

In [2]:
table_name = "sync_dms_dv"
table_temp = "sync_dms_dv_temp"

In [1]:
# print(sql)
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
# df['inserted_At']
# bq_values_insert(df, f"{table_temp}", 3)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_name};
# CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
# PARTITION BY DATE(crtd_datetime)
# CLUSTER BY branchid,batnbr,ordernbr
# """
# print(sql)
# execute_bq_query(sql)

dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)

bq_values_insert(df, f"{table_name}", 2)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_temp};
# """
# execute_bq_query(sql)


In [ ]:
#UPDATE
df_update = get_ms_df(usql)
df_update.columns = lower_col(df_update)
# df_update.columns
df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
df_update1 = df_update['crtd_datetime']
df_update1.drop_duplicates(inplace=True)
tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
# tpl_dt
df_update1 = df_update['branchid'] + df_update['batnbr'] + df_update['ordernbr'] + df_update['sequence']
df_update1.drop_duplicates(inplace=True)
tpl_pk = tuple(df_update1.to_list()) + ('',)
del_sql = \
f"""
DELETE FROM biteam.{table_name}
WHERE
DATE(crtd_datetime) in {tpl_dt}
AND concat(branchid,batnbr,ordernbr,sequence) in {tpl_pk}
"""
print("del_sql ",del_sql)
execute_bq_query(del_sql)
df_update['inserted_at'] = datetime.now()
bq_values_insert(df_update, f"{table_name}", 2)

In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update